# Генерация синтетического датасета
В этом ноутбуке мы разберем генерацию синтетического датасета с помощью библиотеки `giskard`
Библиотека `giskard` берет каждый документ в нашей векторной БД и генерирует вопросы по каждому документу с помощью 6 разных способов. (Мы будем использовать 5)
Этот датасет мы будем использовать в дальнейшей оценке нашего RAG

In [8]:
!pip install "giskard[llm]" pandas gigachain-community unstructured

You should consider upgrading via the '/Users/mikelarg/Projects/gigachain_telegram_bot/.venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

Загружаем документы

In [9]:
from giskard.rag import generate_testset, KnowledgeBase
from langchain_community.document_loaders import UnstructuredMarkdownLoader
import os
import tqdm
import pandas as pd
markdown_path = "../db_builder/mdx_docs/"

docuemnts = []
# Iterate all mdx file in subdirectories
for root, dirs, files in tqdm.tqdm(os.walk(markdown_path)):
    for file in files:
        if file.endswith(".mdx"):
            loader = UnstructuredMarkdownLoader(os.path.join(root, file))
            docuemnts.append(loader.load())
print(f"Total documents: {len(docuemnts)}")

18it [00:07,  2.48it/s]

Total documents: 53


Нарезаем их

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs_list = [item for sublist in docuemnts for item in sublist]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=16000, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)
print(f"Documents splited. Count: {len(doc_splits)}")

Documents splited. Count: 56


Берем первые 15 документов (не будем брать все — этот датасет нужен пока только для показа)

In [11]:
df = pd.DataFrame([d.page_content for d in doc_splits[:15]], columns=["text"])
df

,text
0,Как обновить GigaChain?\n\nВыполните команду b...
1,Общие понятия\n\nclient_id - идентификатор про...
2,GigaChat. Python-библиотека для GigaChain\n\nБ...
3,🦜️🔗 GigaChain (GigaChat + LangChain)\n\nБиблио...
4,Главная\n\nБаза знаний\n\nМедитация от страха ...
5,ИНТЕГРИРУЙТЕ ИИ В СВОЙ БИЗНЕС ================...
6,"напарник в работе, учёбе и дома ==============..."
7,"Не работает GigaChat? Проверьте, что отключен ..."
8,Вход в GigaChat\n\nПосле этого вы увидите окно...
9,Главная\n\nБаза знаний\n\nБаза знаний\n\nБАЗА ...


In [12]:
import giskard
giskard.llm.set_llm_model("gpt-4o")

Генерируем датасет

In [13]:
from giskard.rag.question_generators import complex_questions, double_questions, distracting_questions, situational_questions, simple_questions
knowledge_base = KnowledgeBase.from_pandas(df)

testset = generate_testset(
    knowledge_base,
    question_generators=[simple_questions, complex_questions, double_questions, distracting_questions, situational_questions],
    num_questions=40,
    language='ru',  # optional, we'll auto detect if not provided
    agent_description="Агент по технической поддержке большой языковой модели GigaChat и фреймворка GigaChain", # helps generating better questions
)

2024-09-06 16:06:44,925 pid:4465 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


/Users/mikelarg/Projects/gigachain_telegram_bot/.venv/lib/python3.9/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2024-09-06 16:06:53,972 pid:4465 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


Generating questions:   0%|          | 0/40 [00:00<?, ?it/s]

Сохраняем датасет

In [19]:
df = testset.to_pandas()

In [20]:
df

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
10733a5f-9a97-4def-a994-68f9edf44da5,Какие модели доступны для генерации в GigaChat?,"Для генерации доступны модели: GigaChat Lite, ...",Document 11: lastUpdateDate: 5 августа 2024\n\...,[],"{'question_type': 'simple', 'seed_document_id'..."
2af539e9-a78a-4bde-953c-4c794d5c58d7,Как долго действителен токен доступа для GigaC...,Токен действителен в течение 30 минут.,Document 10: lastUpdateDate: 19 июля 2024\n\ni...,[],"{'question_type': 'simple', 'seed_document_id'..."
c04713dc-7002-42da-a6b9-0809d7c912cb,Как создать проект GigaChat API для физических...,Для создания проекта GigaChat API для физическ...,Document 10: lastUpdateDate: 19 июля 2024\n\ni...,[],"{'question_type': 'simple', 'seed_document_id'..."
f2fe07e2-3c80-4448-b46f-21c7ba4df54b,Какие языки поддерживает GigaChat?,GigaChat поддерживает русский и английский языки.,Document 11: lastUpdateDate: 5 августа 2024\n\...,[],"{'question_type': 'simple', 'seed_document_id'..."
16a1b9a7-da66-4292-8b0d-d9e6a6c7b404,Что такое GigaChain и какие возможности он пре...,GigaChain это фреймворк для разработки приложе...,Document 3: 🦜️🔗 GigaChain (GigaChat + LangChai...,[],"{'question_type': 'simple', 'seed_document_id'..."
c181c8d4-84db-40a0-bdc7-1e67d48d18a7,Какое количество потоков доступно физическим л...,Физическим лицам для работы с API доступен оди...,Document 12: lastUpdateDate: 27 февраля 2024\n...,[],"{'question_type': 'simple', 'seed_document_id'..."
49d583f6-0cf2-4773-9e62-4e5bb8d4dff0,Как можно войти в GigaChat?,Войти в GigaChat можно двумя способами — по те...,Document 8: Вход в GigaChat\n\nПосле этого вы ...,[],"{'question_type': 'simple', 'seed_document_id'..."
8fa3b156-11eb-41a0-8caa-b902b7132808,Какие задачи может выполнять GigaChat в бизнесе?,GigaChat может выполнять различные задачи в би...,Document 9: Главная\n\nБаза знаний\n\nБаза зна...,[],"{'question_type': 'simple', 'seed_document_id'..."
9a5c5e8a-c7b5-4267-9bad-6166e33eb4d0,Какие конкретные механизмы медитации способств...,Медитация — это оздоровительная психологическа...,Document 4: Главная\n\nБаза знаний\n\nМедитаци...,[],"{'question_type': 'complex', 'seed_document_id..."


In [16]:
df.to_csv("dataset.csv")

Ура! Мы сделали синтетический датасет. Теперь нужно его просмотреть, почистить и переходить к следующим шагам:
1. [Создание датасета в GigaLogger](gigalogger_create_dataset.ipynb)
2. [Прогон датасета и получение оценок ответам](evaluation.ipynb)